In [1]:
import os
import pandas as pd

data_root = 'homework/data'  # Шлях до папки з даними
activities = ['idle', 'running', 'stairs', 'walking']
data = []

for activity in activities:
    activity_path = os.path.join(data_root, activity)
    activity_files = os.listdir(activity_path)
    
    for file_name in activity_files:
        file_path = os.path.join(activity_path, file_name)
        df = pd.read_csv(file_path)
        df['activity'] = activity
        data.append(df)

# Об'єднання даних в один датасет
dataset = pd.concat(data, ignore_index=True)
print(dataset.head())


   accelerometer_X  accelerometer_Y  accelerometer_Z activity
0         1.000776         4.616021         8.576031     idle
1         0.718261         4.209007         8.446744     idle
2        -0.909797        -0.282516         9.203311     idle
3         5.099650         0.148441         8.418014     idle
4         1.762132        -0.162806         9.251195     idle


In [2]:
import tsfresh.feature_extraction.feature_calculators as calc
import numpy as np

time_features = []

for activity in activities:
    activity_data = dataset[dataset['activity'] == activity].drop(columns=['activity'])
    activity_time_features = activity_data.apply(lambda column: [calc.mean(column),
                                                                 calc.variance(column),
                                                                 calc.standard_deviation(column),
                                                                 calc.maximum(column),
                                                                 calc.minimum(column),
                                                                 np.argmax(column),
                                                                 np.argmin(column),
                                                                 calc.median(column),
                                                                 np.median(np.abs(column - np.median(column))),  # Медіанне абсолютне відхилення
                                                                 np.percentile(column, 75) - np.percentile(column, 25),  # Міжквартильний діапазон
                                                                 calc.kurtosis(column),
                                                                 calc.root_mean_square(column),
                                                                 calc.skewness(column)], axis=0)
    activity_time_features = np.concatenate(activity_time_features.to_numpy(), axis=0)
    time_features.append(activity_time_features)

time_features_df = pd.DataFrame(time_features, columns=[f"{col}_time_{stat}" for col in dataset.columns[:-1] for stat in ["mean", "variance", "standard_deviation", "maximum", "minimum", "argmax", "argmin", "median", "median_abs_deviation", "interquartile_range", "kurtosis", "root_mean_square", "skewness"]])
dataset = pd.concat([dataset, time_features_df], axis=1)

# Зберегти набір даних з часовими ознаками
dataset.to_csv('activity_dataset_with_time_features.csv', index=False)

In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.impute import SimpleImputer  # Додано імпорт

# Зчитування підготовленого датасету з часовими ознаками
dataset = pd.read_csv('activity_dataset_with_time_features.csv')

# Визначення ознак (X) та цільової змінної (y)
X = dataset.drop(columns=['activity'])
y = dataset['activity']

# Створення об'єкту SimpleImputer
imputer = SimpleImputer(strategy='mean')

# Заміна відсутніх значень в X
X_imputed = imputer.fit_transform(X)

# Розділення на навчальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Побудова та навчання моделей
svm_model = SVC(kernel='linear', max_iter=10000)
svm_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(max_depth=10, random_state=42)  
rf_model.fit(X_train, y_train)

# Оцінка результатів
svm_predictions = svm_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)

svm_accuracy = accuracy_score(y_test, svm_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)

svm_f1_macro = f1_score(y_test, svm_predictions, average='macro')
svm_f1_micro = f1_score(y_test, svm_predictions, average='micro')
rf_f1_macro = f1_score(y_test, rf_predictions, average='macro')
rf_f1_micro = f1_score(y_test, rf_predictions, average='micro')

print("SVM Accuracy:", svm_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("SVM Macro F1 Score:", svm_f1_macro)
print("SVM Micro F1 Score:", svm_f1_micro)
print("Random Forest Macro F1 Score:", rf_f1_macro)
print("Random Forest Micro F1 Score:", rf_f1_micro)


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Accuracy: 0.664448571133808
Random Forest Accuracy: 0.9224698235840297
SVM Macro F1 Score: 0.5037141382982335
SVM Micro F1 Score: 0.664448571133808
Random Forest Macro F1 Score: 0.7269255587216501
Random Forest Micro F1 Score: 0.9224698235840296


In [7]:
from sklearn.impute import SimpleImputer

# Зчитування підготовленого датасету з часовими ознаками
dataset = pd.read_csv('activity_dataset_with_time_features.csv')

# Визначення ознак (X) та цільової змінної (y)
X = dataset.drop(columns=['activity'])
y = dataset['activity']

# Заміна відсутніх значень на середні значення
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Розділення на навчальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Побудова та навчання моделі SVM з налаштуванням гіперпараметрів
svm_model = SVC(kernel='linear', C=1.0, gamma='scale', max_iter=10000)
svm_model.fit(X_train, y_train)

# Побудова та навчання моделі Random Forest
rf_model = RandomForestClassifier(max_depth=10, random_state=42)  
rf_model.fit(X_train, y_train)

# Оцінка результатів
svm_predictions = svm_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)

# Розрахунок метрик
svm_accuracy = accuracy_score(y_test, svm_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)

svm_f1_macro = f1_score(y_test, svm_predictions, average='macro')
svm_f1_micro = f1_score(y_test, svm_predictions, average='micro')
rf_f1_macro = f1_score(y_test, rf_predictions, average='macro')
rf_f1_micro = f1_score(y_test, rf_predictions, average='micro')

print("SVM Accuracy:", svm_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("SVM Macro F1 Score:", svm_f1_macro)
print("SVM Micro F1 Score:", svm_f1_micro)
print("Random Forest Macro F1 Score:", rf_f1_macro)
print("Random Forest Micro F1 Score:", rf_f1_micro)


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Accuracy: 0.664448571133808
Random Forest Accuracy: 0.9224698235840297
SVM Macro F1 Score: 0.5037141382982335
SVM Micro F1 Score: 0.664448571133808
Random Forest Macro F1 Score: 0.7269255587216501
Random Forest Micro F1 Score: 0.9224698235840296


In [8]:
from sklearn.preprocessing import StandardScaler

# Зчитування підготовленого датасету з часовими ознаками
dataset = pd.read_csv('activity_dataset_with_time_features.csv')

# Визначення ознак (X) та цільової змінної (y)
X = dataset.drop(columns=['activity'])
y = dataset['activity']

# Заміна відсутніх значень на середні значення
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Стандартизація вхідних ознак
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Розділення на навчальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Побудова та навчання моделі SVM з налаштуванням гіперпараметрів
svm_model = SVC(kernel='linear', C=1.0, gamma='scale', max_iter=10000)
svm_model.fit(X_train, y_train)

# Побудова та навчання моделі Random Forest
rf_model = RandomForestClassifier(max_depth=10, random_state=42)  
rf_model.fit(X_train, y_train)

# Оцінка результатів
svm_predictions = svm_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)

# Розрахунок метрик
svm_accuracy = accuracy_score(y_test, svm_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)

svm_f1_macro = f1_score(y_test, svm_predictions, average='macro')
svm_f1_micro = f1_score(y_test, svm_predictions, average='micro')
rf_f1_macro = f1_score(y_test, rf_predictions, average='macro')
rf_f1_micro = f1_score(y_test, rf_predictions, average='micro')

print("SVM Accuracy:", svm_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("SVM Macro F1 Score:", svm_f1_macro)
print("SVM Micro F1 Score:", svm_f1_micro)
print("Random Forest Macro F1 Score:", rf_f1_macro)
print("Random Forest Micro F1 Score:", rf_f1_micro)


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Accuracy: 0.4693851232848447
Random Forest Accuracy: 0.9224698235840297
SVM Macro F1 Score: 0.26017770582958355
SVM Micro F1 Score: 0.4693851232848447
Random Forest Macro F1 Score: 0.7269255587216501
Random Forest Micro F1 Score: 0.9224698235840296


In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Зчитування підготовленого датасету з часовими ознаками
dataset = pd.read_csv('activity_dataset_with_time_features.csv')

# Визначення ознак (X) та цільової змінної (y)
X = dataset.drop(columns=['activity'])
y = dataset['activity']

# Заміна відсутніх значень на середні значення
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Стандартизація вхідних ознак
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Побудова та навчання моделі SVM з налаштуванням гіперпараметрів
svm_model = SVC(kernel='linear', C=10000, gamma='scale', max_iter=20000)
svm_model.fit(X_scaled, y)  # Навчання моделі

# Крос-валідація моделі SVM
svm_scores = cross_val_score(svm_model, X_scaled, y, cv=5, scoring='accuracy')

# Побудова моделі Random Forest
rf_model = RandomForestClassifier(max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Крос-валідація моделі Random Forest
rf_scores = cross_val_score(rf_model, X_scaled, y, cv=5, scoring='accuracy')

# Вивід результатів
print("SVM Cross-Validation Accuracy:", svm_scores.mean())
print("Random Forest Cross-Validation Accuracy:", rf_scores.mean())
print("SVM Accuracy:", accuracy_score(y_test, svm_model.predict(X_test)))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_model.predict(X_test)))
print("SVM Macro F1 Score:", f1_score(y_test, svm_model.predict(X_test), average='macro'))
print("SVM Micro F1 Score:", f1_score(y_test, svm_model.predict(X_test), average='micro'))
print("Random Forest Macro F1 Score:", f1_score(y_test, rf_model.predict(X_test), average='macro'))
print("Random Forest Micro F1 Score:", f1_score(y_test, rf_model.predict(X_test), average='micro'))

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_it

SVM Cross-Validation Accuracy: 0.2593005261528939
Random Forest Cross-Validation Accuracy: 0.8907613741875581
SVM Accuracy: 0.3663984318580419
Random Forest Accuracy: 0.9224698235840297
SVM Macro F1 Score: 0.2543093174823222
SVM Micro F1 Score: 0.3663984318580419
Random Forest Macro F1 Score: 0.7269255587216501
Random Forest Micro F1 Score: 0.9224698235840296


In [14]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Зчитування підготовленого датасету з часовими ознаками
dataset = pd.read_csv('activity_dataset_with_time_features.csv')

# Визначення ознак (X) та цільової змінної (y)
X = dataset.drop(columns=['activity'])
y = dataset['activity']

# Заміна відсутніх значень на середні значення
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Стандартизація вхідних ознак
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Побудова та навчання моделі SVM з налаштуванням гіперпараметрів
svm_model = SVC(kernel='rbf', C=500, gamma='scale', max_iter=5000)
svm_model.fit(X_scaled, y)  # Навчання моделі

# Крос-валідація моделі SVM
svm_scores = cross_val_score(svm_model, X_scaled, y, cv=5, scoring='accuracy')

# Побудова моделі Random Forest
rf_model = RandomForestClassifier(max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Крос-валідація моделі Random Forest
rf_scores = cross_val_score(rf_model, X_scaled, y, cv=5, scoring='accuracy')

# Вивід результатів
print("SVM Cross-Validation Accuracy:", svm_scores.mean())
print("Random Forest Cross-Validation Accuracy:", rf_scores.mean())
print("SVM Accuracy:", accuracy_score(y_test, svm_model.predict(X_test)))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_model.predict(X_test)))
print("SVM Macro F1 Score:", f1_score(y_test, svm_model.predict(X_test), average='macro'))
print("SVM Micro F1 Score:", f1_score(y_test, svm_model.predict(X_test), average='micro'))
print("Random Forest Macro F1 Score:", f1_score(y_test, rf_model.predict(X_test), average='macro'))
print("Random Forest Micro F1 Score:", f1_score(y_test, rf_model.predict(X_test), average='micro'))

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5

SVM Cross-Validation Accuracy: 0.6438770246569689
Random Forest Cross-Validation Accuracy: 0.8907613741875581
SVM Accuracy: 0.7233312699886516
Random Forest Accuracy: 0.9224698235840297
SVM Macro F1 Score: 0.5725400578419001
SVM Micro F1 Score: 0.7233312699886516
Random Forest Macro F1 Score: 0.7269255587216501
Random Forest Micro F1 Score: 0.9224698235840296
